In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#importing libraires
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets,transforms
from torch import nn
from torch.nn import Linear 

In [ ]:
transform = transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
training_datasets = datasets.MNIST(root='',download=True,train=True,transform=transform)
test_datasets = datasets.MNIST(root='',download=True,train=False,transform=transform)
training_dataloader = torch.utils.data.DataLoader(dataset=training_datasets,shuffle=True,batch_size=100)
test_dataloader = torch.utils.data.DataLoader(dataset=test_datasets,shuffle=True,batch_size=100)

In [ ]:
def im_convert(tensor):
    image = tensor.numpy()
    image = image.transpose(1,2,0)
    image = image *np.array(0.5) + np.array(0.5)
    image = image.clip(0,1)
    return image[:,:,0]

In [ ]:
data_iter = iter(training_dataloader)
images,labels = data_iter.next()

fig = plt.figure(figsize=(25,4))

for idx in np.arange(10):
    ax = fig.add_subplot(2,10,idx+1)
    plt.imshow(im_convert(images[idx]))
    ax.set_title([labels[idx].item()])

In [ ]:
# Defining CNN  - Copying Le_Net CNN model arhcitechture
class Le_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.Conv1 = nn.Conv2d(in_channels=1,out_channels=20,kernel_size=5,stride=1) # for square kernel, we can give kernel size as only one number
        self.Conv2 = nn.Conv2d(in_channels=20,out_channels=50,kernel_size=5,stride=1)
        self.FC1 = nn.Linear(in_features=4*4*50,out_features=500)
        self.dropout = nn.Dropout(0.5)
        self.FC2 = nn.Linear(in_features=500,out_features=10)
        
        
    def forward(self,x):
        x = F.relu(self.Conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.Conv2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,4*4*50)
        x = F.relu(self.FC1(x))
        x = self.dropout(x)
        x = self.FC2(x)
        return x
    
    def predict(self,x):
        x = F.relu(self.Conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.Conv2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,4*4*50)
        x = F.relu(self.FC1(x))
        x = self.FC2(x)
        return x

In [ ]:
model = Le_Net()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
epochs = 10
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

for epoch in range(epochs):
    batch_loss = []
    batch_accuracies = []
    for inputs,labels in training_dataloader:
        Y_pred = model.forward(inputs)
        loss = criterion(Y_pred,labels)
        batch_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #batch accuracy
        _,pred = torch.max(Y_pred,1)
        batch_accuracy = torch.sum(pred == labels.data).item()
        batch_accuracies.append(batch_accuracy)
    
    test_batch_loss = []
    test_batch_accuracies = []
    for test_inputs,test_labels in test_dataloader:
        with torch.no_grad():
            predictions = model.predict(test_inputs)
            test_loss = criterion(predictions,test_labels)
            test_batch_loss.append(test_loss.item())
            _,test_pred = torch.max(predictions,1)
            test_batch_accuracy = torch.sum(pred == test_labels.data).item()
            test_batch_accuracies.append(batch_accuracy)

    #for training        
    mean_epochloss = np.array(batch_loss).mean()
    mean_epoch_accuracy = np.array(batch_accuracies).mean()
    #for testing
    test_mean_epochloss = np.array(test_batch_loss).mean()
    test_mean_epoch_accuracy = np.array(test_batch_accuracies).mean()
    print('for epoch {} ,training loss {:.4f}/accuracy {:.4f} , Test loss {:.4f}/accuracy as {:.4f}'.format(epoch,mean_epochloss,mean_epoch_accuracy,test_mean_epochloss,test_mean_epoch_accuracy))
    train_losses.append(mean_epochloss)
    train_accuracies.append(mean_epoch_accuracy)
    test_losses.append(test_mean_epochloss)
    test_accuracies.append(test_mean_epoch_accuracy)
    #training accuracy
    
    

In [ ]:
#loss
plt.plot(range(epochs),train_losses,label = 'train')
plt.plot(range(epochs),test_losses,label ='test')
plt.legend()
plt.show()

In [ ]:
#accuracy plot
plt.plot(range(epochs),train_accuracies,label = 'train')
plt.plot(range(epochs),test_accuracies,label = 'test')
plt.legend()
plt.show()

In [ ]:
torch.argmax(F.softmax(predictions),1)


In [ ]:
test_labels

In [ ]:
#predicting images
import requests
from PIL import Image
from PIL import ImageOps
url = 'https://images.homedepot-static.com/productImages/007164ea-d47e-4f66-8d8c-fd9f621984a2/svn/architectural-mailboxes-house-letters-numbers-3585b-5-64_1000.jpg'
response = requests.get(url,stream = True)
img = Image.open(response.raw)
img = ImageOps.invert(img)
img = img.convert('1')
img = transform(img)
plt.imshow(im_convert(img))


In [ ]:
img = img.view(-1,1,28,28)
prediction = model.predict(img)
_,test_pred = torch.max(prediction,1)
print(test_pred.item())

In [ ]:
#visualising what model has learnt
for name,parameters in model.named_parameters():
    if parameters.requires_grad:
        print(name,parameters.data.size())

In [ ]:
# for i in range(20):
#     plt.imshow(model.Conv1.weight[i,0].data.numpy())


# test_no = 16   
# grid_dim = np.int(np.sqrt(test_no))


fig = plt.figure(figsize=(10,10))
for i in range(20):
    temp = fig.add_subplot(5,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    plt.imshow(model.Conv1.weight[i,0].data.numpy())

fig.show()  

In [ ]:
fig = plt.figure(figsize=(10,10))
for i in range(50):
    temp = fig.add_subplot(5,10,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    plt.imshow(model.Conv2.weight[i,0].data.numpy())

fig.show() 

In [ ]:
model.Conv2.weight.data.size()

In [ ]:
#intermediate outputs
#conv1 outout
sample_image = training_dataloader.dataset.data[2,].numpy()
plt.imshow(sample_image)


In [ ]:
from scipy import signal
#conv1 layer output
      
fig = plt.figure(figsize=(10,10))

for i in range(20):
    temp = fig.add_subplot(5,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel = model.Conv1.weight[i,0].data.numpy()
    out = signal.convolve2d(sample_image,kernel,boundary='symm',mode= 'valid')
    plt.imshow(out)

fig.show()  

In [ ]:
sample_image.shape

In [ ]:
#applying relu to them
def ReLU(x):
    return x * (x > 0)


fig = plt.figure(figsize=(10,10))

for i in range(20):
    temp = fig.add_subplot(5,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel = model.Conv1.weight[i,0].data.numpy()
    out = signal.convolve2d(sample_image,kernel,boundary='symm',mode= 'valid')
    out = ReLU(out)
    plt.imshow(out)

fig.show()  

In [ ]:
out.shape

In [ ]:
#applying max pooling
import skimage.measure

fig = plt.figure(figsize=(10,10))

for i in range(20):
    temp = fig.add_subplot(5,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel = model.Conv1.weight[i,0].data.numpy()
    out = signal.convolve2d(sample_image,kernel,boundary='symm',mode= 'valid')
    out = ReLU(out)
    out = skimage.measure.block_reduce(out, (2,2), np.max)
    plt.imshow(out)

fig.show()  

In [ ]:
out.shape

In [ ]:
kernel2 = model.Conv2.weight[0,].data.numpy()
kernel2 = kernel2.transpose(1,2,0)
kernel2.shape

In [ ]:
#applying conv2 
fig = plt.figure(figsize=(10,10))
#from astropy.convolution import convolve
import scipy

for j in range(50): 
    temp = fig.add_subplot(10,5,j+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel2 = model.Conv2.weight[j,].data.numpy()
    kernel2 = kernel2.transpose(1,2,0)
    #print('shape of kernel',kernel2.shape)
    arr = []
    for i in range(20):
        kernel = model.Conv1.weight[i,0].data.numpy()
        out = signal.convolve2d(sample_image,kernel,boundary='symm',mode= 'valid')
        out = ReLU(out)
        out = skimage.measure.block_reduce(out, (2,2), np.max)
        arr.append(out)

    arr = np.array(arr)
    arr = arr.transpose(1,2,0)
    #print('input shape',arr.shape)
    out2= scipy.signal.convolve(arr,kernel2,mode = 'valid')
    plt.imshow(out2[:,:,0])

fig.show() 


In [ ]:
#relu again
fig = plt.figure(figsize=(10,10))
#from astropy.convolution import convolve
import scipy

for j in range(50): 
    temp = fig.add_subplot(10,5,j+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel2 = model.Conv2.weight[j,].data.numpy()
    kernel2 = kernel2.transpose(1,2,0)
    #print('shape of kernel',kernel2.shape)
    arr = []
    for i in range(20):
        kernel = model.Conv1.weight[i,0].data.numpy()
        out = signal.convolve2d(sample_image,kernel,boundary='symm',mode= 'valid')
        out = ReLU(out)
        out = skimage.measure.block_reduce(out, (2,2), np.max)
        arr.append(out)

    arr = np.array(arr)
    arr = arr.transpose(1,2,0)
    #print('input shape',arr.shape)
    out2= scipy.signal.convolve(arr,kernel2,mode = 'valid')
    out2 = ReLU(out2)
    plt.imshow(out2[:,:,0])

fig.show() 


In [ ]:
x = np.array(range(20))
y = np.sin(6*x)
plt.plot(x,y)
plt.show()

### LeNet model on CIFAR10 Dataset

In [ ]:
#importing libraires
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets,transforms
from torch import nn
from torch.nn import Linear 

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  

In [ ]:
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
training_datasets = datasets.CIFAR10(root='./data',download=True,train=True,transform=transform)
test_datasets = datasets.CIFAR10(root='./data',download=True,train=False,transform=transform)
training_dataloader = torch.utils.data.DataLoader(dataset=training_datasets,shuffle=True,batch_size=100)
test_dataloader = torch.utils.data.DataLoader(dataset=test_datasets,shuffle=True,batch_size=100)

In [ ]:
def im_convert(tensor):
    image = tensor.numpy()
    image = image.transpose(1,2,0)
    image = image *np.array(0.5) + np.array(0.5)
    image = image.clip(0,1)
    #image = np.fliplr(image.reshape(-1,3)).reshape(image.shape)
    #print(image.shape)
    return image

In [ ]:
%matplotlib inline 
data_iter = iter(training_dataloader)
images,labels = data_iter.next()

fig = plt.figure(figsize=(25,4))

for idx in np.arange(10):
    ax = fig.add_subplot(2,10,idx+1)
    im = im_convert(images[idx])
    plt.imshow(im_convert(images[idx]))
    ax.set_title([classes[labels[idx].item()]])

In [ ]:
# Defining CNN  - Copying Le_Net CNN model arhcitechture
class Le_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.Conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=1,padding=1) # for square kernel, we can give kernel size as only one number
        self.Conv2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.Conv3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
        self.FC1 = nn.Linear(in_features=4*4*64,out_features=500)
        self.dropout = nn.Dropout(0.5)
        self.FC2 = nn.Linear(in_features=500,out_features=10)
        
        
    def forward(self,x):
        x = F.relu(self.Conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.Conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.Conv3(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,4*4*64)
        x = F.relu(self.FC1(x))
        x = self.dropout(x)
        x = self.FC2(x)
        return x
    
    def predict(self,x):
        x = F.relu(self.Conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.Conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.Conv3(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,4*4*64)
        x = F.relu(self.FC1(x))
        x = self.FC2(x)
        return x

In [ ]:
model = Le_Net()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
epochs = 10
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

for epoch in range(epochs):
    batch_loss = []
    batch_accuracies = []
    for inputs,labels in training_dataloader:
        Y_pred = model.forward(inputs)
        loss = criterion(Y_pred,labels)
        batch_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #batch accuracy
        _,pred = torch.max(Y_pred,1)
        batch_accuracy = torch.sum(pred == labels.data).item()
        batch_accuracies.append(batch_accuracy)
    
    test_batch_loss = []
    test_batch_accuracies = []
    for test_inputs,test_labels in test_dataloader:
        with torch.no_grad():
            predictions = model.predict(test_inputs)
            test_loss = criterion(predictions,test_labels)
            test_batch_loss.append(test_loss.item())
            _,test_pred = torch.max(predictions,1)
            test_batch_accuracy = torch.sum(pred == test_labels.data).item()
            test_batch_accuracies.append(batch_accuracy)

    #for training        
    mean_epochloss = np.array(batch_loss).mean()
    mean_epoch_accuracy = np.array(batch_accuracies).mean()
    #for testing
    test_mean_epochloss = np.array(test_batch_loss).mean()
    test_mean_epoch_accuracy = np.array(test_batch_accuracies).mean()
    print('for epoch {} ,training loss {:.4f}/accuracy {:.4f} , Test loss {:.4f}/accuracy as {:.4f}'.format(epoch+1,mean_epochloss,mean_epoch_accuracy,test_mean_epochloss,test_mean_epoch_accuracy))
    train_losses.append(mean_epochloss)
    train_accuracies.append(mean_epoch_accuracy)
    test_losses.append(test_mean_epochloss)
    test_accuracies.append(test_mean_epoch_accuracy)
    #training accuracy
    
    

In [ ]:
#loss
plt.plot(range(epochs),train_losses,label = 'train')
plt.plot(range(epochs),test_losses,label ='test')
plt.legend()
plt.show()

In [ ]:
#accuracy plot
plt.plot(range(epochs),train_accuracies,label = 'train')
plt.plot(range(epochs),test_accuracies,label = 'test')
plt.legend()
plt.show()

In [ ]:
torch.argmax(F.softmax(predictions),1)


In [ ]:
test_labels

In [ ]:
#predicting images
import requests
from PIL import Image
from PIL import ImageOps
url = 'https://images.unsplash.com/photo-1518791841217-8f162f1e1131?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80'
response = requests.get(url,stream = True)
img = Image.open(response.raw)
#img = ImageOps.invert(img)
#img = img.convert('1')
img = transform(img)
print(img.shape)
plt.imshow(im_convert(img))


In [ ]:
img = img.view(-1,3,32,32)
prediction = model.predict(img)
_,test_pred = torch.max(prediction,1)
print(classes[test_pred.item()])

In [ ]:
#visualising what model has learnt
for name,parameters in model.named_parameters():
    if parameters.requires_grad:
        print(name,parameters.data.size())

In [ ]:
# for i in range(20):
#     plt.imshow(model.Conv1.weight[i,0].data.numpy())


# test_no = 16   
# grid_dim = np.int(np.sqrt(test_no))


fig = plt.figure(figsize=(10,10))
for i in range(16):
    temp = fig.add_subplot(4,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    plt.imshow(model.Conv1.weight[i,].data.numpy())

fig.show()  

In [ ]:
fig = plt.figure(figsize=(10,10))
for i in range(32):
    temp = fig.add_subplot(8,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    img = model.Conv2.weight[i,].data.numpy()
    plt.imshow(img)

fig.show() 

In [ ]:
fig = plt.figure(figsize=(10,10))
for i in range(64):
    temp = fig.add_subplot(8,8,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    img = model.Conv3.weight[i,].data.numpy()
    plt.imshow(img)

fig.show() 

In [ ]:
model.Conv2.weight.data.size()

In [ ]:
#intermediate outputs
#conv1 outout
sample_image = training_dataloader.dataset.data[12,]
plt.imshow(sample_image)


In [ ]:
from scipy import signal
#conv1 layer output
      
fig = plt.figure(figsize=(10,10))

for i in range(16):
    temp = fig.add_subplot(4,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel = model.Conv1.weight[i,].data.numpy()
    #print(kernel.shape)
    #print(sample_image.shape)
    out = signal.convolve(sample_image,kernel,mode = 'valid')
    #print(out.shape)
    plt.imshow(out[:,:,0])

fig.show()  

In [ ]:

from scipy import signal
#relu
#applying relu to them
def ReLU(x):
    return x * (x > 0)
     
fig = plt.figure(figsize=(10,10))

for i in range(16):
    temp = fig.add_subplot(4,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel = model.Conv1.weight[i,].data.numpy()
    #print(kernel.shape)
    #print(sample_image.shape)
    out = signal.convolve(sample_image,kernel,mode = 'valid')
    #print(out.shape)
    out = ReLU(out)
    plt.imshow(out[:,:,0])

fig.show()  

In [ ]:
#applying max pooling
import skimage.measure

from scipy import signal
#relu
#applying relu to them
def ReLU(x):
    return x * (x > 0)
     
fig = plt.figure(figsize=(10,10))

for i in range(16):
    temp = fig.add_subplot(4,4,i+1)
    temp.set_xticks([])
    temp.set_yticks([])
    kernel = model.Conv1.weight[i,].data.numpy()
    #print(kernel.shape)
    #print(sample_image.shape)
    out = signal.convolve(sample_image,kernel,mode = 'valid')
    #print(out.shape)
    out = ReLU(out)
    #print(out.shape)
    out = skimage.measure.block_reduce(out[:,:,0], (2,2), np.max)
    #print(out.shape)
    plt.imshow(out)

fig.show()  

In [ ]:
kernel2 = model.Conv2.weight[0,].data.numpy()
kernel2 = kernel2.transpose(1,2,0)
kernel2.shape

In [ ]:
#applying conv2 
#fig = plt.figure(figsize=(10,10))
from astropy.convolution import convolve
import scipy

for j in range(50): 
    kernel2 = model.Conv2.weight[j,].data.numpy()
    print('shape of kernel',kernel2.shape)
    arr = []
    for i in range(20):
        kernel = model.Conv1.weight[i,0].data.numpy()
        out = signal.convolve2d(sample_image,kernel,boundary='symm',mode= 'valid')
        out = ReLU(out)
        out = skimage.measure.block_reduce(out, (2,2), np.max)
        arr.append(out)

    arr = np.array(arr)
    print('array shape',arr.shape)
#     arr = arr.transpose(1,2,0)
#     kernel2 = kernel2.transpose(1,2,0)
    out2= scipy.ndimage.convolve(out,kernel2,'same')
    print('out2 shape',out2.shape)


In [ ]:
x = np.array(range(20))
y = np.sin(6*x)
plt.plot(x,y)
plt.show()

### Pretrained Models

In [ ]:
#importing libraires
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets,transforms,models
from torch import nn
from torch.nn import Linear 

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  

In [ ]:
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
training_datasets = datasets.CIFAR10(root='./data',download=True,train=True,transform=transform)
test_datasets = datasets.CIFAR10(root='./data',download=True,train=False,transform=transform)
training_dataloader = torch.utils.data.DataLoader(dataset=training_datasets,shuffle=True,batch_size=100)
test_dataloader = torch.utils.data.DataLoader(dataset=test_datasets,shuffle=True,batch_size=100)

In [ ]:
def im_convert(tensor):
    image = tensor.numpy()
    image = image.transpose(1,2,0)
    image = image *np.array(0.5) + np.array(0.5)
    image = image.clip(0,1)
    #image = np.fliplr(image.reshape(-1,3)).reshape(image.shape)
    #print(image.shape)
    return image

In [ ]:
%matplotlib inline 
data_iter = iter(training_dataloader)
images,labels = data_iter.next()

fig = plt.figure(figsize=(25,4))

for idx in np.arange(10):
    ax = fig.add_subplot(2,10,idx+1)
    im = im_convert(images[idx])
    plt.imshow(im_convert(images[idx]))
    ax.set_title([classes[labels[idx].item()]])

In [ ]:
# # Defining CNN  - Copying Le_Net CNN model arhcitechture
# class Le_Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.Conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=1,padding=1) # for square kernel, we can give kernel size as only one number
#         self.Conv2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1)
#         self.Conv3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
#         self.FC1 = nn.Linear(in_features=4*4*64,out_features=500)
#         self.dropout = nn.Dropout(0.5)
#         self.FC2 = nn.Linear(in_features=500,out_features=10)
        
        
#     def forward(self,x):
#         x = F.relu(self.Conv1(x))
#         x = F.max_pool2d(x,2,2)
#         x = F.relu(self.Conv2(x))
#         x = F.max_pool2d(x,2,2)
#         x = F.relu(self.Conv3(x))
#         x = F.max_pool2d(x,2,2)
#         x = x.view(-1,4*4*64)
#         x = F.relu(self.FC1(x))
#         x = self.dropout(x)
#         x = self.FC2(x)
#         return x
    
#     def predict(self,x):
#         x = F.relu(self.Conv1(x))
#         x = F.max_pool2d(x,2,2)
#         x = F.relu(self.Conv2(x))
#         x = F.max_pool2d(x,2,2)
#         x = F.relu(self.Conv3(x))
#         x = F.max_pool2d(x,2,2)
#         x = x.view(-1,4*4*64)
#         x = F.relu(self.FC1(x))
#         x = self.FC2(x)
#         return x

In [ ]:
model = models.vgg16(pretrained=True)

In [ ]:
for params in model.features.parameters():
    params.requires_grad = False
    

In [ ]:
n_input = model.classifier[6].in_features
last_layer = nn.Linear(in_features = n_input,out_features=10)
model.classifier[6] = last_layer

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
epochs = 5
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

for epoch in range(epochs):
    batch_loss = []
    batch_accuracies = []
    for inputs,labels in training_dataloader:
        Y_pred = model.forward(inputs)
        loss = criterion(Y_pred,labels)
        batch_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #batch accuracy
        _,pred = torch.max(Y_pred,1)
        batch_accuracy = torch.sum(pred == labels.data).item()
        batch_accuracies.append(batch_accuracy)
    
    test_batch_loss = []
    test_batch_accuracies = []
    for test_inputs,test_labels in test_dataloader:
        with torch.no_grad():
            predictions = model.predict(test_inputs)
            test_loss = criterion(predictions,test_labels)
            test_batch_loss.append(test_loss.item())
            _,test_pred = torch.max(predictions,1)
            test_batch_accuracy = torch.sum(pred == test_labels.data).item()
            test_batch_accuracies.append(batch_accuracy)

    #for training        
    mean_epochloss = np.array(batch_loss).mean()
    mean_epoch_accuracy = np.array(batch_accuracies).mean()
    #for testing
    test_mean_epochloss = np.array(test_batch_loss).mean()
    test_mean_epoch_accuracy = np.array(test_batch_accuracies).mean()
    print('for epoch {} ,training loss {:.4f}/accuracy {:.4f} , Test loss {:.4f}/accuracy as {:.4f}'.format(epoch+1,mean_epochloss,mean_epoch_accuracy,test_mean_epochloss,test_mean_epoch_accuracy))
    train_losses.append(mean_epochloss)
    train_accuracies.append(mean_epoch_accuracy)
    test_losses.append(test_mean_epochloss)
    test_accuracies.append(test_mean_epoch_accuracy)
    #training accuracy
    
    

In [ ]:
#loss
plt.plot(range(epochs),train_losses,label = 'train')
plt.plot(range(epochs),test_losses,label ='test')
plt.legend()
plt.show()

In [ ]:
#accuracy plot
plt.plot(range(epochs),train_accuracies,label = 'train')
plt.plot(range(epochs),test_accuracies,label = 'test')
plt.legend()
plt.show()